<a href="https://colab.research.google.com/github/raissacamilla23/mythesis/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 548 kB 6.1 MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=96445 sha256=b3680b66a93e5b4e85d0e517d84f5a42ee76bbeb9b20f8bb7e632ac9651993ab
  Stored in directory: /Users/yaojiaqi/Library/Caches/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491096 sha256=1a76b164bac8aaae61e218ec78bac582599c488849ecba372b8740091425dbc4
  Stored in directory: /Users/yaojiaqi/Library/Caches/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32056 sha256=06455319912f545b33a8a26b080a0dd6e3167e17072bb7e2f94413287bbb4131
  Stored in direc

In [ ]:
import numpy as np
import pandas as pd
import re
import sklearn
import nltk
import gensim
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import LatentDirichletAllocation
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaMulticore

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yaojiaqi/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# 1. Data cleaning and splitting

In [ ]:
#import dataset, remove the 'link' column and no match' rows
song_data = pd.read_csv("subsongdata_57650.csv")
song_data = song_data[['artist','song','text','explicit_label']]
song_data = song_data.loc[song_data['explicit_label'] != 'no match']

In [ ]:
#remove'\n' from the lyrics
re_drop = re.compile(r'\n')
song_data[['text']] = song_data[['text']].applymap(lambda x:re_drop.sub(' ',x))
song_data.iloc[0:10]

,artist,song,text,explicit_label
1,ABBA,"Andante, Andante","Take it easy with me, please Touch me gently...",False
2,ABBA,As Good As New,I'll never know why I had to go Why I had to...,False
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,False
7,ABBA,Chiquitita,"Chiquitita, tell me what's wrong You're ench...",False
11,ABBA,Dancing Queen,"You can dance, you can jive, having the time o...",False
12,ABBA,Disillusion,"Changing, moving in a circle I can see your ...",False
13,ABBA,Does Your Mother Know,"You're so hot, teasing me So you're blue but...",False
15,ABBA,Dum Dum Diddle,"I can hear how you work, practising hard Pla...",False
18,ABBA,Fernando,Can you hear the drums Fernando? I remember ...,False
21,ABBA,From A Twinkling Star To A Passing Angel,"Twinkle, Twinkle little star How I wonder wh...",False


## Create explicit songs dataset: song_data_1

In [ ]:
#extract all the rows with explicit_label = True
song_data_1 = song_data.loc[song_data['explicit_label'] == 'True']
song_data_1.iloc[0:10]

,artist,song,text,explicit_label
128,Adam Sandler,Mr. Bake-O,I'm sitting in my chair watching the ...,True
129,Adam Sandler,Steve Polychronopolous,I'm a big fuckin' dick I'm a pain in your as...,True
909,Avril Lavigne,Here's To Never Growing Up,Singing Radiohead at the top of our lungs Wi...,True
929,Avril Lavigne,Rock N Roll,Let them know that we're still rock n roll I...,True
2089,Bruno Mars,Treasure,"Give me all, give me all, give me all your att...",True
2108,Cake,Alpha Beta Parking Lot,Standing in the Alpha Beta parking lot Watch...,True
2116,Cake,Hem Of Your Garment,I have a heart that's made of wood And I am ...,True
2132,Cake,Satan Is My Motor,I've got wheels of polished steel I've tires...,True
2134,Cake,Sheep Go To Heaven,I'm not feeling alright today I'm not feelin...,True
2144,Cake,You Turn The Screws,ou turn the screws You tear down the bridge ...,True


## Create non-explicit songs dataset: song_data_0

In [ ]:
#ramdomly extract 4068 rows with explicit_label = False, which is 3 times as many as song_data_1
song_data_0 = song_data.loc[song_data['explicit_label'] == 'False']
song_data_0 = song_data_0.sample(n=4068, replace=False, random_state=100)
song_data_0.iloc[0:10]

,artist,song,text,explicit_label
51337,Santana,Somewhere In Heaven,Somewhere In heaven There is a place Wai...,False
28056,Cliff Richard,Can't Take The Hurt Anymore,Now that the past has ended My life's an ope...,False
46109,Nitty Gritty Dirt Band,Stand A Little Rain,Dark cloud is coming Headed straight for you...,False
19827,U2,Dirty Day,I don't know you And you don't know the half...,False
44161,Misfits,Only Make Believe,People see us everywhere They think you real...,False
44208,Modern Talking,Diamonds Never Made A Lady,Oh my heart is crying in the night for you O...,False
25069,Bee Gees,Lion In Winter,Lion in my winter. I'm a lion with no crown....,False
22957,Adele,He Won't Go,"Some say, I'll be better without you, They d...",False
29537,Depeche Mode,Leave In Silence,I've told myself so many times before But th...,False
18982,Sting,Dienda,How like the fall To be gone in a day Just...,False


# 2. Topic Modeling

## Data processing: lemmatization & stemming

In [ ]:
len(STOPWORDS)
stopwords = set(STOPWORDS)
stopwords.update(["yeah", "oh", "ya", "let"])
len(stopwords)

341

In [ ]:
def lemmatization(text):
    wnl = WordNetLemmatizer()
    lemmatization = wnl.lemmatize(text, pos='n')
    return lemmatization

In [ ]:
def lemmatization_stemming(text):
    wnl = WordNetLemmatizer()
    lemmatization = wnl.lemmatize(text, pos='n')

    stemmer = PorterStemmer()
    stemming = stemmer.stem(lemmatization)
    return stemming

In [ ]:
def text_preprocess(text):
    result = []
    for token in simple_preprocess(text):
        if token not in stopwords and len(token) > 3:
            result.append(lemmatization_stemming(token))
    return result

In [ ]:
processed_lyrics = song_data_1['text'].map(text_preprocess)
processed_lyrics

128      [sit, chair, watch, plenti, crazi, shit, frien...
129      [fuckin, dick, pain, drink, beer, slice, charl...
909      [sing, radiohead, lung, boom, blare, fall, lov...
929      [know, rock, roll, care, makeup, like, better,...
2089     [attent, babi, tell, littl, wonder, flawless, ...
2108     [stand, alpha, beta, park, watch, forget, watc...
2116     [heart, wood, bide, time, recit, memor, line, ...
2132     [wheel, polish, steel, tire, grab, road, seat,...
2134     [feel, alright, today, feel, great, catch, tod...
2144     [turn, screw, tear, bridg, flimsi, busi, like,...
2711     [turn, know, safe, right, lay, face, face, gir...
2724     [think, waterb, sheet, soak, glad, gotta, work...
2725     [hook, need, nigga, section, bitch, bitch, bit...
2766     [vers, littl, mama, heard, lost, come, love, h...
2783     [vers, time, time, tell, chang, right, probabl...
2790     [vers, liquor, get, thicker, help, stare, girl...
2893     [bitter, mayb, littl, know, appl, worm, singl,.

In [ ]:
dictionary = Dictionary(processed_lyrics)
dictionary.filter_extremes()

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_lyrics]
#bow_corpus

In [ ]:
tfidf_model = TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

## LDA for explicit songs

In [ ]:
lda_model3 = gensim.models.ldamodel.LdaModel(tfidf_corpus, num_topics=2, id2word=dictionary)
for idx, topic in lda_model3.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.003*"nigga" + 0.003*"choru" + 0.003*"love" + 0.003*"girl" + 0.003*"gonna" + 0.003*"want" + 0.003*"babi" + 0.003*"shit" + 0.002*"bitch" + 0.002*"gotta"
Topic: 1 
Words: 0.005*"nigga" + 0.003*"love" + 0.003*"bitch" + 0.003*"right" + 0.003*"feel" + 0.002*"want" + 0.002*"life" + 0.002*"time" + 0.002*"come" + 0.002*"babi"


In [ ]:
pyLDAvis.enable_notebook()
id2word = dictionary
vis = pyLDAvis.gensim.prepare(lda_model3, tfidf_corpus, id2word)
vis

/Users/yaojiaqi/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
1      0.00499  0.0       1        1  52.472515
0     -0.00499  0.0       2        1  47.527481, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
123   Default  15.000000       gonna  15.000000  30.0000  30.0000
331   Default  13.000000       gotta  13.000000  29.0000  29.0000
8     Default   8.000000        came   8.000000  28.0000  28.0000
411   Default  21.000000       choru  21.000000  27.0000  27.0000
1183  Default   4.000000        gimm   4.000000  26.0000  26.0000
129   Default  13.000000        live  13.000000  25.0000  25.0000
179   Default  10.000000        walk  10.000000  24.0000  24.0000
585   Default   8.000000        drop   8.000000  23.0000  23.0000
10    Default   7.000000       crazi   7.000000  22.0000  22.0000
770   Default   2.000000        kany   2.000000  21.0000  21.0000
87    Default   7.000000        pain   7.000000  20.0000  20.0000
204   Default  12.000000        kill  12.000000  19.0000  19.0000
195   Default  17.000000       right  17.000000  18.0000  18.0000
892   Default   6.000000       comin   6.000000  17.0000  17.0000
290   Default   9.000000        bout   9.000000  16.0000  16.0000
225   Default  11.000000        mind  11.000000  15.0000  15.0000
89    Default   9.000000       parti   9.000000  14.0000  14.0000
170   Default  20.000000        girl  20.000000  13.0000  13.0000
112   Default   6.000000        boom   6.000000  12.0000  12.0000
74    Default  11.000000      fuckin  11.000000  11.0000  11.0000
464   Default   6.000000       music   6.000000  10.0000  10.0000
2177  Default   4.000000     goddamn   4.000000   9.0000   9.0000
365   Default   3.000000        dude   3.000000   8.0000   8.0000
81    Default  10.000000        leav  10.000000   7.0000   7.0000
314   Default  14.000000       thing  14.000000   6.0000   6.0000
54    Default   7.000000        told   7.000000   5.0000   5.0000
568   Default   5.000000       south   5.000000   4.0000   4.0000
410   Default   9.000000        wait   9.000000   3.0000   3.0000
258   Default   8.000000        work   8.000000   2.0000   2.0000
41    Default  14.000000        said  14.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
225    Topic2   6.756045        mind  11.395622   0.2211  -6.3595
74     Topic2   6.709146      fuckin  11.533563   0.2021  -6.3665
170    Topic2  11.133853        girl  20.009052   0.1577  -5.8600
54     Topic2   4.751020        told   7.937168   0.2307  -6.7116
81     Topic2   6.235229        leav  10.694263   0.2044  -6.4397
314    Topic2   8.210439       thing  14.613076   0.1674  -6.1645
41     Topic2   8.018859        said  14.434666   0.1560  -6.1881
410    Topic2   5.619464        wait   9.689791   0.1990  -6.5437
258    Topic2   5.151924        work   8.796752   0.2088  -6.6306
39     Topic2   6.026615        play  10.558592   0.1831  -6.4738
42     Topic2   9.863876        shit  18.749428   0.1016  -5.9811
150    Topic2   7.482112       littl  13.686047   0.1400  -6.2574
164    Topic2   9.960838        babi  19.230047   0.0860  -5.9713
20     Topic2   7.151892        head  13.061703   0.1416  -6.3026
83     Topic2   6.405957  motherfuck  11.503112   0.1585  -6.4127
409    Topic2   6.438896        vers  11.701436   0.1465  -6.4076
57     Topic2  10.259677        want  20.840361   0.0352  -5.9417
637    Topic2   5.324173        burn   9.347826   0.1810  -6.5977
130    Topic2  11.328021        love  25.704044  -0.0755  -5.8427
386    Topic2  12.882963       nigga  33.334923  -0.2068  -5.7140
321    Topic2   9.328659       bitch  21.050346  -0.0700  -6.0369
166    Topic2   8.183949        come  17.591871  -0.0214  -6.1678
180    Topic2   7.758246       wanna  16.235123   0.0054  -6.2212
200    Topic2   7.996830        feel  19.013308  -0.1222  -6.1909
161

## LDA for non-explicit songs

In [ ]:
stopwords = set(STOPWORDS)
#stopwords.update(bad_words)
stopwords.update(["yeah","oh", "ya", "let"])
len(stopwords)

341

In [ ]:
def text_preprocess(text):
    result = []
    for token in simple_preprocess(text):
        if token not in stopwords and len(token) > 3:
            result.append(lemmatization_stemming(token))
    return result

In [ ]:
processed_normal_lyrics = song_data_0['text'].map(text_preprocess)

normal_dictionary = Dictionary(processed_normal_lyrics)
normal_dictionary.filter_extremes()

bow_corpus_normal = [normal_dictionary.doc2bow(doc) for doc in processed_normal_lyrics]

tfidf_model_normal = TfidfModel(bow_corpus_normal)
tfidf_corpus_normal = tfidf_model_normal[bow_corpus_normal]

lda_model_normal = gensim.models.ldamodel.LdaModel(tfidf_corpus_normal, num_topics=4, id2word=normal_dictionary)

for idx, topic in lda_model_normal.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.009*"safe" + 0.007*"fortress" + 0.006*"goodby" + 0.006*"distanc" + 0.006*"feel" + 0.006*"love" + 0.006*"sacr" + 0.006*"chain" + 0.005*"mari" + 0.005*"tearin"
Topic: 1 
Words: 0.008*"love" + 0.007*"want" + 0.006*"good" + 0.005*"like" + 0.005*"littl" + 0.005*"come" + 0.005*"tonight" + 0.004*"gone" + 0.004*"babi" + 0.004*"caus"
Topic: 2 
Words: 0.008*"lisa" + 0.007*"lost" + 0.007*"awhil" + 0.007*"mighti" + 0.005*"lift" + 0.005*"travelin" + 0.005*"pave" + 0.004*"burn" + 0.004*"walk" + 0.004*"want"
Topic: 3 
Words: 0.007*"tomorrow" + 0.007*"eddi" + 0.006*"jesu" + 0.006*"love" + 0.005*"thousand" + 0.005*"temptat" + 0.005*"year" + 0.005*"choru" + 0.004*"think" + 0.004*"matter"


In [ ]:
pyLDAvis.enable_notebook()
id2word = normal_dictionary
vis = pyLDAvis.gensim.prepare(lda_model_normal, tfidf_corpus_normal, id2word)
vis

/Users/yaojiaqi/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.090254 -0.001246       1        1  46.153961
2      0.041352 -0.051722       2        1  19.983566
3      0.017793 -0.028431       3        1  18.824327
0      0.031109  0.081398       4        1  15.038141, topic_info=     Category       Freq      Term       Total  loglift  logprob
term                                                            
1524  Default  29.000000      lisa   29.000000  30.0000  30.0000
922   Default  30.000000      safe   30.000000  29.0000  29.0000
1370  Default  29.000000     awhil   29.000000  28.0000  28.0000
3208  Default  25.000000      eddi   25.000000  27.0000  27.0000
1461  Default  22.000000  fortress   22.000000  26.0000  26.0000
937   Default  30.000000    mighti   30.000000  25.0000  25.0000
1731  Default  20.000000  travelin   20.000000  24.0000  24.0000
390   Default  46.000000  tomorrow   46.000000  23.0000  23.0000
565   Default  21.000000   distanc   21.000000  22.0000  22.0000
1543  Default  17.000000      sacr   17.000000  21.0000  21.0000
785   Default  28.000000    goodby   28.000000  20.0000  20.0000
2555  Default  16.000000    tearin   16.000000  19.0000  19.0000
2996  Default  15.000000      puso   15.000000  18.0000  18.0000
3025  Default  15.000000      wala   15.000000  17.0000  17.0000
3323  Default  19.000000      pave   19.000000  16.0000  16.0000
1019  Default  24.000000     chain   24.000000  15.0000  15.0000
638   Default  18.000000      mari   18.000000  14.0000  14.0000
1432  Default  31.000000      lift   31.000000  13.0000  13.0000
1424  Default  22.000000   temptat   22.000000  12.0000  12.0000
28    Default  51.000000      lost   51.000000  11.0000  11.0000
866   Default  31.000000      jesu   31.000000  10.0000  10.0000
1565  Default  41.000000  thousand   41.000000   9.0000   9.0000
251   Default  31.000000      stay   31.000000   8.0000   8.0000
1276  Default  16.000000      park   16.000000   7.0000   7.0000
10    Default  20.000000    anymor   20.000000   6.0000   6.0000
754   Default  14.000000    sunday   14.000000   5.0000   5.0000
160   Default  17.000000   sunshin   17.000000   4.0000   4.0000
49    Default  15.000000      darl   15.000000   3.0000   3.0000
1209  Default  19.000000      cure   19.000000   2.0000   2.0000
792   Default  25.000000     style   25.000000   1.0000   1.0000
...       ...        ...       ...         ...      ...      ...
2010   Topic4   3.252514     bench    4.353516   1.6030  -6.7755
2835   Topic4   3.903445     porch    5.244476   1.5993  -6.5931
746    Topic4   4.076971       ray    5.477960   1.5992  -6.5496
1609   Topic4   6.266628       sea    8.509506   1.5886  -6.1197
754    Topic4  10.306468    sunday   14.405993   1.5597  -5.6222
49     Topic4  10.702873      darl   15.444019   1.5279  -5.5844
941    Topic4   6.075624      toss    8.421877   1.5680  -6.1507
1019   Topic4  15.731908     chain   24.445854   1.4538  -5.1993
785    Topic4  17.517193    goodby   28.573837   1.4053  -5.0918
160    Topic4  11.131286   sunshin   17.657070   1.4332  -5.5452
745    Topic4   6.765936     quiet    9.694016   1.5350  -6.0430
10     Topic4  11.861011    anymor   20.272694   1.3586  -5.4817
1495   Topic4   9.359653      dawn   15.500498   1.3901  -5.7185
251    Topic4  14.199402      stay   31.637486   1.0934  -5.3017
40     Topic4   9.472941   alright   17.291615   1.2928  -5.7065
1209   Topic4   9.624261      cure   19.686611   1.1789  -5.6907
460    Topic4   6.947874   endless   11.365128   1.4025  -6.0165
131    Topic4  16.578211      feel   78.089546   0.3448  -5.1469
141    Topic4   8.994810      warm   20.398771   1.0758  -5.7583
29     Topic4  16.530659      love  119.215683  -0.0811  -5.1497
758    Topic4  10.174472      wish   32.855301   0.7223  -5.6351
9      Topic4  10.752283      wait   39.919678   0.5828  -5.5798
242    Topic4  10.347158      long   40.536942   0.5291  -